In [1]:
import os
import re
import pandas as pd
from datetime import datetime

In [2]:
directory = "C:/Users/Science Direct Climate Risk Management Text/"

In [3]:
data = {'filename': [], 'title': [], 'abstract': [], 'received_date': [], 'revised_date': [], 'accepted_date': []}

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            
            # extract title and author
            title_start = text.find("journal homepage: www.elsevier.com/locate/crm")
            title_end1 = text.find("A R T I C L E", title_start)
            title_end2 = text.find("A B S T R A C T", title_start)
            title_end3 = text.find('Introduction', title_start)
            if title_end1 != -1:
                title = text[title_start:title_end1].strip().replace("journal homepage: www.elsevier.com/locate/crm", "")
            elif title_end2 != -1:
                title = text[title_start:title_end2].strip().replace("journal homepage: www.elsevier.com/locate/crm", "")
            elif title_end3 != -1:
                title = text[title_start:title_end3].strip().replace("journal homepage: www.elsevier.com/locate/crm", "")
            else:
                title = "N/A"
            
            # extract abstract
            start_abstract = text.find('A B S T R A C T')
            end_abstract1 = text.find('Introduction', start_abstract)

            if end_abstract1 != -1:
                # ABSTRACT is before INTRODUCTION
                abstract = text[start_abstract:end_abstract1].replace('A B S T R A C T', '')
            else:
                # ABSTRACT is after INTRODUCTION
                text = text.replace('A B S T R A C T\n\n', '')
                end_abstract2 = text.find('\n\n', start_abstract)
                abstract = text[start_abstract:end_abstract2].replace('A B S T R A C T', '')

            # extract date
            pattern = r"Received\s+\d{1,2}\s+\w+\s+\d{4};\s+Received in revised form\s+\d{1,2}\s+\w+\s+\d{4};\s+Accepted\s+\d{1,2}\s+\w+\s+\d{4}"
            date_match = re.search(pattern, text)
            if date_match:
                match_str = date_match.group(0)
                match_list = match_str.split("; ")
                received_date = re.search('\d{1,2}\s+\w+\s+\d{4}', match_list[0]).group(0)
                revised_date = re.search('\d{1,2}\s+\w+\s+\d{4}', match_list[1]).group(0)
                accepted_date = re.search('\d{1,2}\s+\w+\s+\d{4}', match_list[2]).group(0)
                
            else:
                received_date = ''
                revised_date = ''
                accepted_date = ''

            # add data to dataframe
            data['filename'].append(filename)
            data['title'].append(title)
            data['abstract'].append(abstract)
            data['received_date'].append(received_date)
            data['revised_date'].append(revised_date)
            data['accepted_date'].append(accepted_date)
            
df = pd.DataFrame(data)
df[['received_date', 'revised_date', 'accepted_date']] = df[['received_date', 'revised_date', 'accepted_date']].apply(pd.to_datetime, dayfirst=True)
df.to_excel('Science Direct Climate Risk Management Articles.xlsx')